Solution and Notebook made by Riccardo Cardona [(GitHub link)](https://github.com/Riden15) and Nicholas Berardo [(GitHub link)](https://github.com/Niiikkkk)

# LAB3

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [19]:
from random import choices,randint, choice
import random

import lab3_lib

In [20]:
fitness = lab3_lib.make_problem(10)
for n in range(10):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

10100111010010000111001110111010000110010101110111: 15.33%
11000110011001101011001101110010110100110110001101: 15.33%
01001101001110001000110111110001011110000001100011: 15.33%
01100010110101101011000110001101011000110101100101: 9.11%
01100110110110001000100101100100000000110101110110: 29.56%
11100010000001110001100101001111110001111100110100: 7.33%
00101101110011000100100101110100101011110011110010: 15.33%
11110101001101001001100110100000101010011011110111: 15.33%
11000000101010011110101111110001000110111111000110: 15.33%
01101001011111110001001011000010101010100011010011: 9.13%
10


## EA
- ``mutation``: function that creates a new individual. In this case, we simply change the random value of the genotype. If the selected number of the genome is 0, it will become 1 and viceversa.
- ``one_cut_xover``: takes two individuals and produces one. The one-cut technique was used, i.e., a random value is chosen which will be the index of the cut of the parent genotypes.
- ``xover``: it creates a child genome by randomly selecting each gene from either parent with equal probability.

In [21]:
def mutation(genome):
    index = randint(0,len(genome[0])-1)
    genome[0][index] = 1-genome[0][index]
    return genome[0]

def one_cut_xover(ind1, ind2):
    cut_point = randint(0, len(ind1[0]))
    offspring = ind1[0][:cut_point]+ind2[0][cut_point:]
    return offspring

def xover(genome1, genome2):
    child_genome = [g1 if random.random() > 0.5 else g2 for g1, g2 in zip(genome1[0], genome2[0])]
    return child_genome

## Population functions
- ``init_population``: function that creates the population. An individual is a tuple with a ``length`` elemento long array of zero or one and the fitness of this array. 
- ``gen_new_population``: function that, given the population, returns a new population with the addition of some individual created with ``mutation`` or ``xover``. When we add the individual to the population, we calculate its fitness
- ``select_parent``: function that returns the champion out of the population. We first take the better half of the population, then we randomly choose half of these individuals, and then we take the best.
- ``replacement``: function that joins the initial population and the new population to select the best ``POPULATION_SIZE`` individuals

In [22]:
def init_population(n_individual,length,fitness):
    pop = []
    for _ in range(n_individual):
        ind = (choices([0, 1], k=length))
        pop.append((ind,fitness(ind)))
    return pop

def gen_new_population(offspring_size,mutation_prob,old_population,fitness):
    new_individual = []
    for _ in range(offspring_size):
        if random.random() < mutation_prob:
            old_ind = select_parent(old_population)
            tmp = mutation(old_ind)
        else:
            old_ind = select_parent(old_population)
            old_ind_2 = select_parent(old_population)
            tmp = one_cut_xover(old_ind,old_ind_2)
        new_individual.append((tmp,fitness(tmp)))
    return new_individual

def select_parent(population):
    best_parents = sorted(population,key= lambda i:i[1],reverse=True)[:int(len(population)/2)]
    pool = [choice(best_parents) for _ in range(int(len(population)/4))]
    champion = max(pool, key=lambda i: i[1])
    return champion

def replacement(new_pop,old_pop):
    tmp_pop = new_pop + old_pop
    sorted_pop = sorted(tmp_pop,key= lambda i:i[1],reverse=True)
    return sorted_pop[:len(old_pop)]

## Problem parameters
- ``POPULATION_SIZE``: number of individuals in the population
- ``OFFSPRING_SIZE``: number of new individuals created by the ``gen_new_population`` function
- ``LENGTH_INDV``: how many numbers does an individual have
- ``GENERATION``: number of generations of population to create
- ``MUTATION_PROBABILITY``: probability to do mutation instead of crossover

In [23]:
POPULATION_SIZE = 500
OFFSPRING_SIZE = 500
LENGTH_INDV = 1000
GENERATION = 500
MUTATION_PROBABILITY = 0.1

problem_size = [1,2,5,10]

In [24]:
for ps in problem_size:
    fit = lab3_lib.make_problem(ps)
    pop = init_population(POPULATION_SIZE,LENGTH_INDV,fit)
    best = 0
    n_calls = 0
    gen = 0
    for g in range(GENERATION):
        new_pop = gen_new_population(OFFSPRING_SIZE,0.1,pop,fit)
        pop = replacement(new_pop,pop)
        if pop[0][1] > best:
            best = pop[0][1]
            n_calls = fit.calls
            gen = g
    print(f"Problem size: {ps}")
    print(f"Best fitness: {best}")
    print(f"Fitness calls: {n_calls}")
    print(f"Generation: {gen}")
    print(f"Population size: {POPULATION_SIZE}")

Problem size: 1
Best fitness: 1.0
Fitness calls: 160500
Generation: 319
Population size: 500
Problem size: 2
Best fitness: 0.896
Fitness calls: 238000
Generation: 474
Population size: 500
Problem size: 5
Best fitness: 0.465
Fitness calls: 179000
Generation: 356
Population size: 500
Problem size: 10
Best fitness: 0.2719
Fitness calls: 248500
Generation: 495
Population size: 500
